In [1]:
import torch
#import torchvision
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

In [2]:
class Residual(nn.Module):#自定义一个残差网络
    def __init__(self, in_channels, out_channels, use_1x1conv=False, stride=1):
        #初始化时需要对网络的输入通道，输出通道，以及是否使用１＊１卷积，步长等进行设置
        super(Residual, self).__init__()
        #网络中需要使用的卷积层构建
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=stride)
        #构建了一个输入与输出量均为系统要求输出通道的网络conv2
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(out_channels)#归一化，使得同一批输入特征的feature map均满足均值为０，方差为１的分布规律
        self.bn2 = nn.BatchNorm2d(out_channels)#归一化

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))#先卷积再归一化再激活函数
        Y = self.bn2(self.conv2(Y))#在前面的基础上再次卷积，并归一化
        if self.conv3:#由于１＊１卷积不是网络标配，必须进行判断
            X = self.conv3(X)
        return F.relu(Y + X)

In [3]:
blk = Residual(3, 3)#实例化残差网络，输入通道为３，输出通道为３
X = torch.rand((4, 3, 6, 6))#batch=4,通道为３,图像尺度为６＊６
print(blk(X).shape )

torch.Size([4, 3, 6, 6])


In [4]:
blk = Residual(3, 6, use_1x1conv=True, stride=2)#实例化残差网络，输入通道为３，输出通道为6，使用１＊１卷积模块，步长为２
print(blk(X).shape) # torch.Size([4, 6, 3, 3])

torch.Size([4, 6, 3, 3])
